# Geometric Kernels

[GPJax](https://github.com/JaxGaussianProcesses/GPJax) is a Python package for working wit Gaussian processes in JAX. This notebook highlights how Geometric Kernels can be integrated into the functionality provided in GPJax.

In [3]:
import gpjax as gpx
import jax.numpy as jnp
import jax.random as jr
from jax.config import config
import geometric_kernels.jax 
from geometric_kernels.frontends.jax.gpjax import GPJaxGeometricKernel
from geometric_kernels.kernels import MaternKarhunenLoeveKernel
from geometric_kernels.spaces import Mesh
import jax
import meshzoo         

config.update("jax_enable_x64", True)
key = jr.PRNGKey(123)

2023-09-02 13:52:31.444970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
INFO: Using numpy backend


AttributeError: module 'gpjax.kernels' has no attribute 'AbstractKernelComputation'

## Data

We'll now define a dataset that we'll seek to model. The data support used in this example is an icose sphere from the [MeshZoo](https://github.com/meshpro/meshzoo) library. A Matérn kernel is then defined on the sphere and a single draw is taken from the Gaussian process' prior distribution at a random set of points to give us a response variable.

In [2]:
resolution = 3
num_data = 25
vertices, faces = meshzoo.icosa_sphere(resolution)
mesh = Mesh(vertices, faces)

truncation_level = 20
base_kernel = MaternKarhunenLoeveKernel(mesh, truncation_level)
geometric_kernel = GPJaxGeometricKernel(base_kernel)

init_params = geometric_kernel.init_params(key)


def get_data():
    _X = jr.randint(key, minval=0, maxval=mesh.num_vertices, shape=(num_data, 1))
    _K = geometric_kernel.gram(init_params, _X)
    _L = jnp.linalg.cholesky(_K.to_dense() + jnp.eye(_K.shape[0]) * 1e-6)
    _y = _L @ jr.normal(key, (num_data,))
    return _X, _y


X, y = get_data()
X_test = jnp.arange(mesh.num_vertices).reshape(mesh.num_vertices, 1)

## Model specification

A model can now be defined. We'll purposefully keep this section brief as the workflow is identical to that of a regular Gaussian process regression model that is detailed in [full](https://gpjax.readthedocs.io/en/latest/examples/regression.html).

In [3]:
data = gpx.Dataset(X=X, y=y.reshape(-1, 1))

prior = gpx.Prior(kernel=geometric_kernel)
gpx.config.add_parameter("nu", gpx.config.Softplus)

likelihood = gpx.likelihoods.Gaussian(num_datapoints=num_data)

posterior = likelihood * prior

/tmp/ipykernel_5060/1632642032.py:1: DeprecatedWarning: Dataset is deprecated as of 0.5.5 and will be removed in 0.6.0. Use JaxUtils for a Dataset object
  data = gpx.Dataset(X=X, y=y.reshape(-1, 1))
/tmp/ipykernel_5060/1632642032.py:4: DeprecatedWarning: add_parameter is deprecated as of 0.5.6 and will be removed in 0.6.0. Use method from jaxutils.config instead.
  gpx.config.add_parameter("nu", gpx.config.Softplus)


As with a regular conjugate Gaussian process, the marginal log-likelihood is tractable and can be evaluated using the posterior's `marginal_log_likelihood` method.

In [4]:
params, _, _ = gpx.initialise(posterior, key).unpack()

posterior.marginal_log_likelihood(data)(params)

Array(-23.32777922, dtype=float64)

Derivatives of the marginal log-likelihood can be taken.

In [5]:
grads = jax.grad(posterior.marginal_log_likelihood(data, negative=True))(params)
print(grads)

{'kernel': {'lengthscale': Array([-0.52077028], dtype=float64), 'nu': Array([0.], dtype=float64)}, 'likelihood': {'obs_noise': Array([12.17021713], dtype=float64)}, 'mean_function': {}}


Finally, the predictive posterior can be computed for making predictions at unseen points. Evaluating the predictive posterior distribution returns a multivariate Gaussian distribution for which we can compute the posterior mean and variance as follows.

In [6]:
predictive_posterior = posterior.predict(params, data)(X_test)

mu = predictive_posterior.mean()
sigma2 = predictive_posterior.variance()

In [1]:
import jax

In [2]:
jax.__version__

'0.4.13'